# Stock data analysis

In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
from urllib.parse import urlencode
import datetime

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [5]:
class YahooFinanceWebsite:
    def __init__(self, stock_symbol):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.stock_symbol = stock_symbol.upper()

    def __build_url(self, params):
        base_url = f"https://finance.yahoo.com/quote/{self.stock_symbol}/history/"
        query_string = urlencode(params)
        return f"{base_url}?{query_string}"

    def get_stock_data(self):
        datetime_now = datetime.datetime.now()
        datetime_year_ago = datetime_now - datetime.timedelta(days=365)
        params = {"frequency": "1wk", "period1": datetime_year_ago.timestamp(), "period2": datetime_now.timestamp()}
        url = self.__build_url(params)
        response = requests.get(url, headers=headers)

        soup = BeautifulSoup(response.content, 'html.parser')
        
        title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        html_table_data = soup.find("table")

        return title, html_table_data

In [6]:
def build_stock_analysis_prompt(stock_symbol, title, stock_table_data):
    sys_prompt = r"""You are an assistant that analyzes the contents of HTML formated table that contains data on a specific stock.
    The HTML table contains the date, open price, close price, low and highs aggregated for every week over one year timeframe.
    Ignoring text, tags or html attributes that might be navigation related. 
    Respond in Markdown format"""
    
    user_prompt = f"The data provided below in the HTML table format for {stock_symbol} from the Yahoo Finances.\
    Make the explaination easy enough for a newbie to understand. \
    Analyze and Summarize the trends on this stock:\n{stock_table_data}\n\n\
    Also, calculate the total returns in percentage one could have expected over this period."
    
    return [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt}
    ]

In [7]:
def analyze_stock_trends(stock_symbol):
    stock_data_page = YahooFinanceWebsite(stock_symbol)
    title, stock_table_data = stock_data_page.get_stock_data()
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = build_stock_analysis_prompt(stock_symbol, title, stock_table_data)
    )
    return response.choices[0].message.content

def display_analysis(stock_symbol):
    display(Markdown(analyze_stock_trends(stock_symbol)))

In [8]:
display_analysis("GOOG")

# GOOG Stock Trends Analysis

Here's a breakdown of the trends noticed in the provided data for Google's stock (GOOG) over the observed time frame:

## General Observations

1. **Price Fluctuations**:
   - The stock opened at **165.32** on March 14, 2025, and reached a high of **170.16** just a week before.
   - Significant fluctuations can be noted throughout the year, with highs and lows indicating varying investor sentiments and market conditions.

2. **Overall Trend**:
   - The stock has seen both upward and downward trends, but appears to have fluctuated largely within the range of **150-200** during the specified timeframe.
   - Notable highs were recorded, with peaks around **189.99** and **208.70** observed in early February 2025.

3. **Recent Movements**:
   - The stock price saw a substantial drop from levels over **200** in February 2025 to the **165-170** range by mid-March 2025.
   - This behavior might indicate market corrections or reactions to broader economic factors affecting technology stocks.

4. **Volume Changes**:
   - The trading volume over the observed weeks has shown variability. High trading volumes, like **106,375,800** and **139,159,100**, correspond with periods of sharp price changes.
   - A spike in volume is generally an indicator of increased interest and can correlate with significant price movements.

5. **Dividends**:
   - Several dividends were declared during the time, such as **$0.20** on March 10, 2025, which can add slight returns to investors who hold stocks.

## Calculating Total Returns

To find the total returns over this period, we look at the price change from the beginning to the end of the timeframe, following this simple formula:

\[
\text{Total Returns \%} = \left( \frac{\text{Final Price} - \text{Initial Price}}{\text{Initial Price}} \right) \times 100
\]

### Given Data:
- **Initial Price (March 25, 2024)**: 150.95
- **Final Price (March 14, 2025)**: 167.62

### Calculation:
- Total Returns \% = 
\[
\left( \frac{167.62 - 150.95}{150.95} \right) \times 100 \approx \left( \frac{16.67}{150.95} \right) \times 100 \approx 11.04\%
\]

## Conclusion

- **Short-Term Investment Outlook**: Given the above findings, GOOG displayed volatility but showed an approximate **11.04% total return** over this period. This can be considered a reasonable investment return, especially against the backdrop of technology market conditions.
- **Long-Term View**: If you are considering investing in GOOG, while the fluctuations can be alarming in a short term, this stock has historically shown significant growth over longer periods. Understanding market trends and staying informed will be key for successful investing. 

In summary, GOOG presents a fluctuating yet moderately promising investment profile, emphasizing the importance of both timing and patience when investing in the stock market.

In [9]:
display_analysis("PFE")

## Analysis of Pfizer Inc. Stock Data (PFE)

### Overview
The data provided shows weekly stock prices for Pfizer Inc. (PFE) over a specific period, including the opening price, closing price, highest price, lowest price, and trading volume. The aim is to identify trends and calculate the total return on investment.

### Key Metrics to Understand
- **Open Price**: The price of the stock at the market's opening.
- **Close Price**: The price at which the stock closes at the end of trading hours.
- **High Price**: The highest price reached during the trading session.
- **Low Price**: The lowest price during the trading session.
- **Volume**: The number of shares traded during the period.

### Trend Analysis
1. **Overall Trend**:
   - The stock seems to show fluctuating performance over the observed timeframe, with periods of both high and low prices. 
   - Here are some highlighted points:
     - In early March 2025, the stock hovered around the **$26 range**, fluctuating between **$25.50 to $27.24**.
     - A significant drop can be seen around November 2024, where prices dipped to **$24.80 - $25.65** range.
     - The stock price most recently peaked at around **$31.54** in July 2024, indicating the potential for growth within the stock.

2. **Recent Performance**:
   - In late 2024, the stock experienced a drop in prices, with a closing price of **$25.72** in mid-March 2025, indicating a decrease compared to the early 2024 highs.
   - Higher trading volumes were observed during the overall fluctuation periods, indicating investor interest during these price changes, particularly around major peaks and drops.

3. **Dividend Events**:
   - The table indicates dividend payments (e.g., **$0.42** and **$0.43**), which contribute to the total returns for investors receiving dividends, representing an essential aspect of returns on investment.

### Total Return Calculation
To calculate the total return, we use the following formula:

\[
\text{Total Return (\%)} = \left(\frac{\text{Final Value} - \text{Initial Value}}{\text{Initial Value}}\right) \times 100
\]

#### Values from the Data
- **Initial Value** (Earliest close price recorded in the dataset from July 2024): \( \text{Close price on Jul 1, 2024} = 28.05 \)
- **Final Value** (Latest close price recorded on Mar 14, 2025): \( \text{Close price on Mar 14, 2025} = 25.72 \)

\[
\text{Total Return (\%)} = \left(\frac{25.72 - 28.05}{28.05}\right) \times 100
\]
\[
\text{Total Return (\%)} = \left(\frac{-2.33}{28.05}\right) \times 100 \approx -8.31\%
\]

### Conclusion
- **Trend Summary**: The stock saw highs above $30 but has generally changed in a downward trend, culminating in a total return of approximately -8.31%. This indicates a decrease in the stock's value over the observed period.
- **Investment Consideration**: Potential investors should consider the volatility and recent downward trend, coupled with dividend returns, which may provide some cushion against losses.

In summary, PFE's performance had highs and lows, with a notable decline in recent periods, which should be critically evaluated for investment purposes.

In [10]:
display_analysis("AAPL")

# AAPL Stock Trends Analysis

## Summary of Data

The provided data shows the weekly performance of Apple Inc. (AAPL) over recent months. It includes key metrics such as:

- **Open Price**: The price at which the stock started the week.
- **High Price**: The highest price during the week.
- **Low Price**: The lowest price during the week.
- **Close Price**: The price at which the stock closed for the week.
- **Adjusted Close Price**: This reflects the closing price adjusted for dividends and stock splits.
- **Volume**: The total number of shares traded during the week.

### Key Trends:

1. **Price Fluctuations**:
   - Over the most recent weeks, AAPL has shown significant price fluctuations. 
   - The stock opened at **211.25** on **March 14, 2025**, and reached a weekly high of **244.03** on **March 3, 2025**.

2. **Volatility**:
   - AAPL witnessed high volatility with significant swings in prices. For instance, the closing prices fluctuated close to **20%** from week to week.

3. **Recent Highs and Lows**:
   - The stock reached a peak close price of **244.60** on **February 10, 2025** and bottomed at **165.00** on **April 15, 2024**, showing a downward trend in some weeks.

4. **Dividends**:
   - AAPL paid dividends of **$0.25** on several occasions during the observed period, which is a positive sign of financial health and a commitment to returning value to shareholders.

5. **Volume Trends**:
   - There were weeks with very high trading volumes, such as **635.75 million** shares traded on **June 10, 2024**. This can indicate increased investor interest or volatility during those weeks.

### Overall Trend

From a broader perspective, the stock began at approximately **191.51** in late May 2024 and has shown intervals of growth and decline throughout the subsequent months. The stock's recent performance and closing prices indicate that AAPL stock is currently experiencing a rebound, albeit with signs of market volatility.

## Calculation of Total Returns

To calculate the total returns over this period from **May 6, 2024** to **March 14, 2025**:

1. **Start Price (May 6, 2024)**: 183.05
2. **End Price (March 14, 2025)**: 213.49

### Total Return Calculation:

\[
\text{Total Return} = \left( \frac{\text{End Price} - \text{Start Price}}{\text{Start Price}} \right) \times 100
\]

Substituting the values:

\[
\text{Total Return} = \left( \frac{213.49 - 183.05}{183.05} \right) \times 100 = \left( \frac{30.44}{183.05} \right) \times 100 \approx 16.63\%
\]

## Conclusion

Overall, if an investor held AAPL stock from May 6, 2024, to March 14, 2025, they could have expected a total return of approximately **16.63%**, alongside dividends, reflecting reasonable growth in a volatile market. Investors should keep an eye on market trends and stay informed about upcoming earnings reports and market sentiments that may influence stock prices further.